In [25]:
import psycopg2
import nltk
import pandas as pd
import numpy as np
import re
import string
import spacy
import sklearn
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [26]:
# importing and creating df (has to have type_id)

# usage: specify file location, sample size and seed(used by random) 
filepath = '/Users/Master/Documents/KU/2.Semester/Datascience/news_sample.csv' #
#s = 250                                            # desired sample size
#seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0])
content = df['content']

In [38]:
df['content']

0      Sometimes the power of Christmas will make you...
1      AWAKENING OF 12 STRANDS of DNA – “Reconnecting...
2      Never Hike Alone: A Friday the 13th Fan Film U...
3      When a rare shark was caught, scientists were ...
4      Donald Trump has the unnerving ability to abil...
                             ...                        
245    Prison for Rahm, God’s Work And Many Others\n\...
246    4 Useful Items for Your Tiny Home\n\nHeadline:...
247    Former CIA Director Michael Hayden said Thursd...
248    Antonio Sabato Jr. says Hollywood's liberal el...
249    Former U.S. President Bill Clinton on Monday c...
Name: content, Length: 250, dtype: object

In [27]:
nlp = spacy.load('en_core_web_lg')

In [23]:
text = ("these word are features")
with nlp.disable_pipes():
    vectors = np.array([token.vector for token in nlp(text)])

In [7]:
doc = nlp("these word, are features")
for token in doc:
    print(token)
#token.lemma_
#token.is_stop

these
word
,
are
features


In [4]:
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(sentence).vector for sentence in df['content']])

In [66]:
fakeidx = df.index[df.type == 'fake'].tolist()

In [11]:
fakevecs = np.array([doc_vectors[i] for i in fakeidx])

In [54]:

len(fakevecs)
np.linalg.norm(fakevecs[0])
#calc avg vector of the type-vectors:
#np.linalg.norm(fakevecs[29])

2.442973

In [28]:
df.type.fillna('no type', inplace = True)
#X_train, X_test, y_train, y_test = train_test_split(doc_vectors, df.type, test_size = 0.1, random_state= 1)

In [28]:
a = nlp('REPLY NOW FOR FREE TEA').vector
b = nlp('REPLY NOW FOR FREE TEA AND Potatoes').vector

In [16]:
svc = LinearSVC(random_state = 1, dual = False, max_iter = 10000)
svc.fit(X_train, y_train)
print(f"Accuracy: {svc.score(X_test, y_test) * 100:.3f}%",)
#svc.score(X_test,y_test) * 100

Accuracy: 64.000%


In [29]:
#sklearn.metrics.pairwise.cosine_similarity(a,b)
def cosine_similarity(a,b):
    return a.dot(b)/np.sqrt(a.dot(a) * b.dot(b))

In [37]:
cosine_similarity(fakevecs[152],doc_vectors[45])

0.95132565

In [7]:
#try tf-idf svm
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df["content"])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, df.type, test_size = 0.1, random_state= 1)

In [12]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.52

In [80]:
x = nlp("man")
y = nlp("king")
print(x.similarity(y))
print(x.similarity(x))



0.40884613016820903
1.0


In [29]:
import gensim 
from gensim.models.doc2vec import TaggedDocument, Doc2Vec 
from collections import namedtuple

In [31]:
df['type_id'] = df.groupby(['type']).ngroup()

In [32]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
series_content = df.content

contTok =series_content.str.split().tolist()
typeLst =df.type_id.tolist()

#doc1 = ["This is a sentence", "This is another sentence"]
#docs = []
#analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
#for i, text in enumerate(doc1):
#    words = text.lower().split()
#    tags = [i]
#    docs.append(analyzedDocument(words, tags))

In [17]:
#model = Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)

In [33]:
#model = gensim.models.doc2vec.Doc2Vec(vector_size=250, min_count=2, epochs=40)
#documents = [TaggedDocument(contTok,df.index) for sentence in df]
documents = [TaggedDocument(contTok, [i]) for i, contTok in enumerate(contTok)]

In [34]:
model = Doc2Vec(documents, vector_size = 250, min_count = 2, epochs= 40)

In [61]:
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
#doc2 = nlp(df.content[20])v
#tokdoc2 = [token for token in nlp(df.content[20])] 
#for token in doc2:
#    print(token)

In [43]:
v = model.infer_vector(['trump'])

In [56]:
df.type[4]

'clickbait'

In [46]:
sim_doc = model.docvecs.most_similar([12])
sim_doc

[(196, 0.854720950126648),
 (83, 0.7812755703926086),
 (239, 0.7757385969161987),
 (4, 0.7741408348083496),
 (192, 0.7681198716163635),
 (128, 0.7517058849334717),
 (74, 0.734205961227417),
 (110, 0.7236354351043701),
 (66, 0.706433117389679),
 (15, 0.704484224319458)]

In [60]:
#df.loc[df.type == 'bias']

In [59]:
#df.loc[192]

In [78]:
print(model.docvecs.most_similar(contTok[80]))
model.docvecs[5]

TypeError: '<' not supported between instances of 'str' and 'int'

In [32]:
df.content[40]

'Offers are getting into the six-figure range in pre-bidding for the NamesCon live auction\n\nHeadline: Bitcoin & Blockchain Searches Exceed Trump! Blockchain Stocks Are Next!\n\nIn a few weeks domain name investors from around the world will converge in Las Vegas for NamesCon, the largest event in the domain industry. One of the highlights of the conference is the live domain auction hosted by ROTD and pre-bidding has already started on NameJet, and like the title says, there are already six-figure bids.\n\nIf you’ve never been to a live domain auction I can tell you – it’s a rush that’s hard to explain, at least for a domain geek like me. Monte Cahn, one of the most respected domain industry veterans will be running the auction again this year and from what I’ve seen over the years, when Monte’s involved, a lot of the truly premium domains come out.\n\nSo I’m not incredibly surprised that we’re already starting to see six-figure bids come out in pre-bidding more than two weeks before